# Building a Group Chat with AutoGen

In this notebook, we create a multi-agent group chat using the autogen library.

The logical stack for combining multiple agents in a platform requries the following logical stack

![magentic](../../assets/images/logicalstack.png)

This playground is about demonstrating the pattern of dynamic unplanned collaboration between different agent types to solve abstract problems
![magentic](../../assets/images/magentic.png)

## 1. Setting Up Our Environment

We will import the necessary libraries and configure our environment.

In [ ]:
import asyncio
import os
import pytz
from datetime import datetime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv()

## 2. Configuring the Model Client

Set up the model client based on available environment variables.

In [ ]:
if os.environ.get("GITHUB_TOKEN") is None:
    model_client = AzureOpenAIChatCompletionClient(
        azure_deployment=os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
        model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
        model_info={
            "json_output": True,
            "function_calling": True,
            "vision": False,
            "family": "unknown",
        }
    )
else:
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    token = os.environ["GITHUB_TOKEN"]
    endpoint = "https://models.inference.ai.azure.com"
    model_name = "gpt-4o-mini"

    # Create a model client for AutoGen
    model_client = OpenAIChatCompletionClient(
        model=model_name,
        base_url=endpoint,
        api_key=token
    )

## 3. Defining Tools and Agents

We define tools and specialized agents to handle specific tasks such as retrieving usernames, locations, and current time.

In [ ]:
# Define tools
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

def get_current_username(input: str) -> str:
    "Get the username of the current user."
    print("executing get_current_username")
    return "Dennis"

def get_current_location_of_user(username: str) -> str:
    "Get the current timezone location of the user for a given username."
    print("executing get_current_location")
    print(username)
    if "Dennis" in username:
        return "Europe/Berlin"
    else:
        return "America/New_York"

def get_current_time(location: str) -> str:
    "Get the current time in the given location. The pytz is used to get the timezone for that location. Location names should be in a format like America/Seattle, Asia/Bangkok, Europe/London. Anything in Germany should be Europe/Berlin"
    try:
        print("get current time for location: ", location)
        timezone = pytz.timezone(location)
        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")
        return current_time
    except Exception as e:
        print("Error: ", e)
        return "Sorry, I couldn't find the timezone for that location."
    

# Define agents
users_agent = AssistantAgent(
    "users_agent",
    model_client=model_client,
    tools=[get_current_username],
    description="A helpful assistant that can knows things about the user like the username.",
    system_message="You are a helpful assistant that can retrieve the username of the current user.",
)

location_agent = AssistantAgent(
    "location_agent",
    model_client=model_client,
    tools=[get_current_location_of_user],
    description="A assistant that can find the physical location of a user.",
    system_message="You are a helpful assistant that can suggest details for a location and can utilize any context information provided.",
)

time_agent = AssistantAgent(
    "time_agent",
    model_client=model_client,
    tools=[get_current_time],
    description="A helpful assistant that knows time in a specific location.",
    system_message="You are a helpful assistant that can retrieve the current time for a given location.",
)

summary_agent = AssistantAgent(
    "summary_agent",
    model_client=model_client,
    description="A helpful assistant that can summarize details about conversations.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and leverage them to answer questions. You must ensure that you use that the other agents can solve the problem. When all open questions have been answered, you can respond with TERMINATE.",
)

## 4. Running the Group Chat

We now create an interactive function to run the group chat with different queries.

In [ ]:
async def run_group_chat(query: str) -> None:
    """Run the group chat with a specific query"""
    # Set up termination conditions
    inner_termination = MaxMessageTermination(5)
    
    # Create the group chat
    magenticteam = MagenticOneGroupChat(
        [users_agent, location_agent, time_agent],
        model_client=model_client,
        termination_condition=inner_termination
    )
    
    # Run the team and stream messages to the console
    stream = magenticteam.run_stream(task=query)
    await Console(stream)

# Example query to run
await run_group_chat("what time is it here?")

## 5. Interactive Group Chat

Now let's try an interactive version where you can input your own queries:

In [ ]:
async def interactive_chat():
    while True:
        query = input("Enter your query (or 'exit' to quit): ")
        if query.lower() == "exit":
            break
        await run_group_chat(query)
        print("\n---Chat complete---\n")

# Run the interactive chat
await interactive_chat()

## 6. Conclusion

In this notebook, we demonstrated how to build a multi-agent group chat using autogen. You can create additional agents or refine the conversation flow to suit various collaborative tasks.

Try experimenting with different types of questions like:
- "What time is it in New York?"
- "Who am I?"
- "What's my location?"

Each of these questions will engage different agents in the group chat based on their specializations.